In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

In [6]:
examples,metadata=tfds.load('ted_hrlr_translate/pt_to_en',with_info=True,as_supervised=True)
train_examples,val_examples=examples['train'],examples['validation']

W1001 14:42:14.409526 140333418604288 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [7]:
tokenizer_en=tfds.features.text.SubwordTextEncoder.build_from_corpus((en.numpy() for pt,en in train_examples),
                                                                     target_vocab_size=2**13)
tokenizer_pt=tfds.features.text.SubwordTextEncoder.build_from_corpus((pt.numpy() for pt,en in train_examples),
                                                                     target_vocab_size=2**13)

In [9]:
sample_string="Transformers' advantage is that it calculates values at the same time"

tokenized_string=tokenizer_en.encode(sample_string)
print("Tokeenized string is:{}".format(tokenized_string))

original_string=tokenizer_en.decode(tokenized_string)
print("The original string is:{}".format(original_string))

assert original_string==sample_string

Tokeenized string is:[7915, 1248, 7946, 1369, 523, 7870, 7863, 6122, 13, 10, 17, 4119, 9, 5506, 38, 3, 168, 160]
The original string is:Transformers' advantage is that it calculates values at the same time


In [15]:
for ts in tokenized_string:
    print('{} -----> {}'.format(ts,tokenizer_en.decode([ts])))

7915 -----> T
1248 -----> ran
7946 -----> s
1369 -----> form
523 -----> ers
7870 -----> '
7863 ----->  
6122 -----> advantage 
13 -----> is 
10 -----> that 
17 -----> it 
4119 -----> calculate
9 -----> s 
5506 -----> values 
38 -----> at 
3 -----> the 
168 -----> same 
160 -----> time


In [16]:
BUFFER_SIZE=60000
BATCH_SIZE=64

In [19]:
def encode(lang1,lang2):
    lang1=[tokenizer_pt.vocab_size]+tokenizer_pt.encode(lang1.numpy())+[tokenizer_pt.vocab_size+1]
    lang1=[tokenizer_en.vocab_size]+tokenizer_en.encode(lang2.numpy())+[tokenizer_en.vocab_size+1]
    return lang1,lang2

In [20]:
MAX_LENGTH=40

In [21]:
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                          tf.size(y)  <= max_length)

In [22]:
def tf_encode(pt,en):
    return tf.py_function(encode,[pt,en],[tf.int64,tf.int64])

In [26]:
train_dataset=train_examples.map(tf_encode)
train_dataset=train_dataset.filter(filter_max_length)
#cache dataset to the memory to get speedup while readng from it
train_dataset=train_dataset.cache()
train_dataset=train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,padded_shapes=([-1],[-1]))
train_dataset=train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

val_dataset=val_examples.map(tf_encode)
val_dataset=val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE,padded_shapes=([-1],[-1]))

In [32]:
"""pt_batch, en_batch = next(iter(val_dataset))
pt_batch, en_batch"""

'pt_batch, en_batch = next(iter(val_dataset))\npt_batch, en_batch'

In [33]:
val_dataset.shape()

AttributeError: 'PaddedBatchDataset' object has no attribute 'shape'